In [2]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from auto.auto_crawl import get_data1, get_data2, get_data3, get_data4, get_data5, get_data6, get_data7_1, get_data7_2, get_data7_3, get_data7_4, get_data7_5, get_data7_6, get_data8_1, get_data8_2, get_data9_1, get_data9_2, get_data_10, scroll_down
import re
import time
import os
import arrow
from selenium.webdriver.chrome.options import Options
import requests
import undetected_chromedriver as uc

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import tqdm.auto as tqdm
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import matplotlib.pyplot as plt
plt.rc('font', family='Microsoft JhengHei')
plt.rcParams['axes.unicode_minus'] = False # 顯示負號
import seaborn as sns
import os
import operator

import math
from collections import defaultdict
from wordcloud import WordCloud, STOPWORDS

In [8]:
get_data9_2()
file_path = os.listdir('auto/new_data')
file_paths = [path for path in file_path if 'ferrous'in path or 'Ferrous' in path]
filenames = file_paths.copy()
first_time = 0

try:
    updatepath = 'auto/data/'
    path_update = f'{updatepath}9_2原料交易量.json'
    jsonFile = open(path_update,'r')
    f =  jsonFile.read()   # 要先使用 read 讀取檔案
    DF = json.loads(f)
    jsonFile.close()
except:
    first_time = 1
    DF = {}
    DF['Ferrous'] = {}
    DF['Non-ferrous'] = {}

for file_path in file_paths:
    df = pd.read_excel(f'auto/new_data/{file_path}', index_col=0)
    cleaned_filename = file_path.replace('9_2_', '').replace('.xlsx', '').split('_')
    if first_time == 1:
        DF[cleaned_filename[0]][cleaned_filename[1]] = {}
    data = DF[cleaned_filename[0]][cleaned_filename[1]]
    for i in range(0,len(df)):
        if i == 0:
            columns = df.columns
            for column in columns:
                if column not in data.keys():
                    data[column] = []
        if df.iloc[i]['date'] in data['date'] and df.iloc[i]['CONTRACT'] == data['CONTRACT'][-1]:
            print(df.iloc[i]['date'])
            continue
        for column in columns:
            data[column].append(str(df.iloc[i][column]))

2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18


In [5]:
df.iloc[i]['date']

'2024/11/18'

In [6]:
data['date']

['2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/13',
 '2024/11/18',
 '2024/11/18']

In [3]:
# 參考字典
tmp_ref = {'世界':['世界'],
'人民幣':['中國', 'RMB', '人幣', '神州', 'Shibor', '央行', '人民幣'],
'美元':['美國', '紐約', 'Fed', 'USD', '美聯儲', '聯準會','美元'],
'台幣':['台灣', '台海', '訪台', 'TWD', '台幣'],
'日圓':['日本', 'JPY', '日圓'],
'澳幣':['澳洲','澳幣'],
'英鎊':['英國','英鎊'],
'歐元':['德國', '法國','歐元']}

value_tmp = list(tmp_ref.values())
key_tmp = list(tmp_ref.keys())

df_ref = pd.DataFrame()
for i in range(0,len(value_tmp)):
    for tmp in value_tmp[i]:
        tmp = pd.DataFrame([key_tmp[i], tmp]).T
        df_ref = pd.concat([df_ref, tmp])
df_ref.columns = ['市場別', '名稱']

df_ref_dict = df_ref['名稱'].values

df_ref_dict = dict(zip(df_ref_dict, [1]*len(df_ref_dict)))

#產業 and 項目
sectors = list(set(df_ref['市場別']))

#公司
companies = df_ref['名稱'].values

dirPath = r"Forex_dashboard/原始鉅亨"
input_files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

# 切詞
ws = WS("Forex_dashboard/data")
pos = POS("Forex_dashboard/data")
ner = NER("Forex_dashboard/data")


# 建立一個詞典，詞典中的每個詞都有一個對應的權重
word_to_weight = df_ref_dict

# 使用 construct_dictionary 函數來將上述的詞與權重建立成詞典
dictionary = construct_dictionary(word_to_weight)

# 一定要切的標點符號
segment_delimiter_set =  {",", "。", ":", "?", "!", ";","〈","〉","，","(",")", " ", "、"}

target_filelist = os.listdir("Forex_dashboard/標完產業與項目的鉅亨_標題與關鍵字")

for filename in tqdm.tqdm(input_files):
    if filename in target_filelist:
        continue
    df = pd.read_csv('Forex_dashboard/原始鉅亨/'+ filename, index_col = '日期')
    df['標題'] = df['標題'].astype(str)
    sentence_list = df['標題'].values

    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        segment_delimiter_set = segment_delimiter_set, # This is the defualt set of delimiters
        # recommend_dictionary = dictionary, # words in this dictionary are encouraged
        coerce_dictionary = dictionary, # words in this dictionary are forced
    )

    pos_sentence_list = pos(word_sentence_list) # 詞性

    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    ## 找標題中的名詞
    related_company_ls = []
    df['標題相關詞彙'] = word_sentence_list
    df['關鍵字'] = [tmp.split(',') if type(tmp)!= float else [''] for tmp in df['關鍵字']]
    df['標題與關鍵字相關詞彙'] = df.apply(lambda x: x['標題相關詞彙']+x['關鍵字'], axis = 1)

    ## 找標題中的名詞的公司 & 其產業
    company_list = []
    sector_list = []
    for i in range(0,len(df)):
        tmp = df['標題與關鍵字相關詞彙'][i]
        tmp_list = []
        tmp_sector_list = []
        for j in range(len(tmp)):
            if tmp[j] in df_ref_dict.keys():
                tmp_list.append(tmp[j])
                tmp_sector_list.append(df_ref[df_ref['名稱'] == tmp[j]]['市場別'].values)
        if len(tmp_list) == 0:
            tmp_list = ['未標示']
            tmp_sector_list = [['未標示']]
        company_list.append(tmp_list)
        sector_list.append(tmp_sector_list)
    df['標題相關公司'] = company_list
    df['標題相關產業'] = sector_list

    # 擴增, 有些標題有許多家公司
    DF = pd.DataFrame()
    for i in range(0,len(df)):
        company_list = df['標題相關公司'].iloc[i]
        sector_list = df['標題相關產業'].iloc[i]
        tmp = df.iloc[i].copy()
        if len(company_list) > 0:
            for j in range(0,len(company_list)):
                tmp['標題相關公司'] = company_list[j]
                tmp['標題相關產業'] = sector_list[j][0]
                DF = pd.concat([DF,tmp], axis = 1)
        else:
            DF = pd.concat([DF,tmp], axis = 1)
    DF = DF.T

    ## 將產業轉dummy
    for sector in sectors:
        DF[sector] = DF['標題相關產業'].apply(lambda x: 1 if sector in x else 0)

    DF.to_csv('./Forex_dashboard/標完產業與項目的鉅亨_標題與關鍵字/'+filename, encoding = 'utf-8-sig', index = True)
# 釋放 WS、POS、NER 使用的記憶體
del ws
del pos
del ner


 99%|█████████▉| 259/261 [00:00<00:00, 7860.13it/s]


ValueError: Index 日期 invalid

In [5]:
filename

'cnyes-2024-11-01~2024-11-16.csv'

In [6]:
pd.read_csv('Forex_dashboard/原始鉅亨/'+ filename)

,2024-11-15,18:05,"美匯指數報106.548,關注美國10月工業及零售數據",0,外匯,Unnamed: 5,Unnamed: 6,美匯指數 ==== 最新美元匯價 ====== *上述報價只供參考用,https://news.cnyes.com/news/id/5781268
0,2024-11-15,17:18,台灣續列匯率操縱觀察名單 央行回應美方2項建議,1,台股,台股新聞,"匯率,央行,美元,台幣,貿易順差,經常帳",美國財政部公布半年度匯率政策報告 (檢視期間為 2023 年下半年至 2024 年上半年)，...,https://news.cnyes.com/news/id/5781113
1,2024-11-15,17:06,"《神州經脈》消費增速勝預期,樓價環比跌幅縮,滬指周挫逾3%",0,A股港股,大陸政經,NaN,"《神州經脈》國家統計局今日(15日)發布的一系列經濟數據有好有壞,滬綜指震盪調整,全日收跌1...",https://news.cnyes.com/news/id/5781159
2,2024-11-15,17:05,"英鎊兌美元1.2663,英第三季GDP按季增長0.1%遜預期",0,外匯,NaN,NaN,美匯指數 ==== 最新美元匯價 ...,https://news.cnyes.com/news/id/5781147
3,2024-11-15,16:44,"【聚焦人幣】人幣兌一美元即期收升99點子,報7.2310",0,A股港股,大陸政經,NaN,"《經濟通通訊社15日專訊》兌一美元即期匯率(CNY)今日收報7.2310,較上個交易日4時3...",https://news.cnyes.com/news/id/5781089
4,2024-11-15,16:40,"《財資快訊》美電升報7.7833,美元兌人幣軟報7.2313",0,A股港股,港股,NaN,《財資快訊》美電軟報7.7818╱7847。美元兌升報7.2311╱2314。一年期不交收益...,https://news.cnyes.com/news/id/5781083
...,...,...,...,...,...,...,...,...,...
215,2024-11-01,09:37,"【聚焦人幣】人幣兌一美元即期開市升21點子,報7.1139",0,A股港股,大陸政經,NaN,"《經濟通通訊社1日專訊》兌一美元中間價今報7.1135,為10月25日以來最高,較上個交易日...",https://news.cnyes.com/news/id/5760155
216,2024-11-01,09:21,"【聚焦人幣】人幣中間價升115點子,報7.1135",0,外匯,NaN,NaN,"《經濟通通訊社1日專訊》中國外匯交易中心兌一美元中間價今報7.1135,為10月25日以來最...",https://news.cnyes.com/news/id/5760143
217,2024-11-01,09:03,"歐元兌美元報1.0887,歐元區10月CPI按年升2.0%",0,外匯,NaN,NaN,美匯指數 ==== 最新美元匯價 ...,https://news.cnyes.com/news/id/5760071
218,2024-11-01,08:59,【遠東商銀】關注美國就業報告，美元漲跌互見,0,外匯,NaN,NaN,◆USD/JPY美國第三季國內生產總值 (GDP) 環比增長年率為 2.8%、消費者支出以一...,https://news.cnyes.com/news/id/5749315


In [2]:
url = 'https://www.macromicro.me/charts/117199/global-share-of-international-payments-via-swift-by-major-currency'
driver = uc.Chrome()
driver.get(url)
time.sleep(5)
data = driver.find_elements(By.XPATH,'//*[@id="ccApp"]/div/div[2]/div[1]/div/div/div[2]/div[4]/ul')[0].text.split()
df = {
    '幣別':[],
    '最新公告時間':[],
    '占比':[],
    # '前一期':[]
}
for i in range(0,len(data), 4):
    df['幣別'].append(data[i])
    df['最新公告時間'].append(data[i+1])
    df['占比'].append(data[i+2])
    # df['前一期'].append(data[i+3])

df = pd.DataFrame(df)
test = [float(tmp[:-1]) for tmp in df.iloc[1:]['占比']]
df.iloc[0]['占比'] = str(round(100-np.sum(test),2))+'%'
df.iloc[0]['最新公告時間'] = df.iloc[1]['最新公告時間']

In [139]:
jsonFile = open("auto/data/12_SWIFT各幣別支付占比.json",'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df12 = json.loads(f)      # 再使用 loads
jsonFile.close()
df12_labels = list(tmp for tmp in df12.keys())

In [144]:
df12['人民幣']['時間'][-1]

'2024-09'

In [146]:
df12['人民幣']['SWIFT占比'][-1]

'3.61%'

In [133]:
updatepath = 'auto/data/'
path_update = f'{updatepath}9_1原料價格.json'
jsonFile = open(path_update,'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df_update = json.loads(f)
jsonFile.close()

In [137]:
df_update['Non-ferrous']['LME-Aluminium'].keys()

dict_keys(['LME Aluminium Official Prices', 'LME Aluminium Closing Prices', 'LME Aluminium opening stocks', 'LME Aluminium Asian Reference Price'])

In [138]:
df_update['Non-ferrous']['LME-Aluminium']['LME Aluminium Official Prices']

{'date': ['2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14'],
 'CONTRACT': ['Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27',
  'Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27',
  'Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27'],
 'BID': ['2528.00',
  '2564.00',
  '2608.00',
  '2643.00',
  '2657.00',
  '2513.50',
  '2549.00',
  '2593.00',
  '2628.00',
  '2653.00',
  '2503.00',
  '2524.00',
  '2572.00',
  '2617.00',
  '2652.00'],
 'OFFER': ['2530.00',
  '2565.00',
  '2613.00',
  '2648.00',
  '2662.00',
  '2514.00',
  '2549.50',
  '2598.00',
  '2633.00',
  '2658.00',
  '2504.00',
  '2526.00',
  '2577.00',
  '2622.00',
  '2657.00']}

In [127]:
querys = ['Currency', 'Currency_date','Currency_unit', 'Stock Market', 'Stock Market_date', 'Stock Market_unit']
for key in list(data.keys()):
    for query in querys:
        data[key][f'{query}'] = data[key][f'{query}'][-2:]

In [129]:
json_data = json.dumps(data)
with open(path_update, "w") as json_file:
    json_file.write(json_data)

In [116]:
now = arrow.now()

# 轉換成指定格式 Sep/24 時:分:秒
formatted_time = now.format("DD/MMM/YY HH:mm:ss")

print(formatted_time)

15/Nov/24 14:42:51


In [113]:
# updated = request.form.get('updated_item')  # Retrieve updated_item from the form
updated = 'all'

GEOs = ['china', 'india', 'malaysia', 'turkey', 'united-states']
filepath = 'auto/new_data/'

for GEO in GEOs:
    options = Options() 
    # options.add_argument('--headless=new')  # 啟動Headless 無頭
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options = options)
    driver.get(f'https://tradingeconomics.com/{GEO}/indicators')
    updated_items = driver.find_elements(By.XPATH, '//*[@id="pagemenutabs"]')[0].text.split()
    time.sleep(3)
    macro = driver.find_elements(By.XPATH, '//*[@id="pagemenutabs"]')[0].text.split('\n')[:-1]
    rows = []
    for l in range(0, len(macro)):
        driver.find_element(By.XPATH, f'//*[@id="pagemenutabs"]/li[{l+1}]/a').click()
        data = driver.find_elements(By.XPATH, f'//*[@id="{macro[l].lower()}"]/div/div/table/tbody')[0].text.replace('30 Year Mortgage Rate', 'thirty Year Mortgage Rate').replace('15 Year Mortgage Rate', 'fifteen Year Mortgage Rate').replace('Michigan 5 Year Inflation Expectations', 'Michigan five Year Inflation Expectations').replace('4-week','four-week').replace('1 Year MLF Rate', 'one Year MLF Rate').replace('14-Day Reverse Repo Rate', 'fourteen-Day Reverse Repo Rate').replace('Loan Prime Rate 5Y', 'Loan Prime Rate five-Y').split('\n')

        for i in range(0,len(data)):
            tmp = data[i].split(' ')
            name = ''
            unit = ''
            notice_date = ''
            last = ''
            previous = ''
            highest = ''
            lowest = ''
            signal = 0
            pre = 0
            for j in range(0,len(tmp)):
                # j = pre
                if signal == 0:
                    if ord(tmp[j][0])>=65: # name
                        name += tmp[j]+' '
                    else: # 數字們
                        signal = 1
                        k = j

                if signal == 1 and ord(tmp[j][0])>=65:
                    if '/' not in tmp[j]:
                        unit += tmp[j]+' '
                    else:
                        signal = 2
                        pre = j
                if signal == 2:
                    if '/' in tmp[j]:
                        notice_date = tmp[j]
                
                
            name = name[:-1]
            unit = unit[:-1]

            try:
                if '/' not in tmp[k+1] and ord(tmp[k+1][0])<65:
                    previous += tmp[k+1]
                else:
                    previous = ''
            except:
                previous = ''
            
            try:
                if '/' not in tmp[k+2] and ord(tmp[k+2][0])<65:
                    highest += tmp[k+2]
            except:
                highest = ''
            
            try:
                if '/' not in tmp[k+3]and ord(tmp[k+3][0])<65:
                    lowest += tmp[k+3]
            except:
                lowest = ''
            
            if 'Stock' in name or 'Currency' in name:
                # 取得當前時間
                now = arrow.now()

                # 轉換成指定格式 Sep/24 時:分:秒
                formatted_time = now.format("MMM/DD HH:mm:ss")

                rows.append({'總經種類':macro[l],'指標名稱':name, 'Last':tmp[k], 'Previous': previous, 'Higest': highest, 'Lowest': lowest, '單位': unit, '公告日期': formatted_time})
            else:    
                rows.append({'總經種類':macro[l],'指標名稱':name, 'Last':tmp[k], 'Previous': previous, 'Higest': highest, 'Lowest': lowest, '單位': unit, '公告日期': notice_date})
    data = pd.DataFrame(rows)

In [114]:
data

,總經種類,指標名稱,Last,Previous,Higest,Lowest,單位,公告日期
0,Overview,Currency,107,107,,,,Nov/15 14:38:07
1,Overview,Stock Market,5927,5949,,,points,Nov/15 14:38:07
2,Overview,GDP Growth Rate,2.8,3,,,percent,Sep/24
3,Overview,GDP Annual Growth Rate,2.7,3,,,percent,Sep/24
4,Overview,Unemployment Rate,4.1,4.1,,,percent,Oct/24
...,...,...,...,...,...,...,...,...
347,Energy,Refinery Crude Runs,175,281,,,Thousand Barrels,Nov/24
348,Health,Hospital Beds,2.77,2.78,,1000,per people,Dec/21
349,Health,Hospitals,18.55,19.11,,,per one million people,Dec/19
350,Health,Medical Doctors,2.77,2.74,,1000,per people,Dec/19


In [3]:
df = get_data4()
df.to_excel('auto/new_data/4_政府推動計畫名單.xlsx', index = 0)

100%|██████████| 89/89 [01:03<00:00,  1.41it/s]


In [ ]:
# df = pd.read_excel('auto/new_data/4_政府推動計畫名單.xlsx')

# try:
#     updatepath = 'auto/data/'
#     path_update = f'{updatepath}4_政府推動計畫名單.xlsx'
#     DF = pd.read_excel(path_update)
# except:
#     DF = pd.DataFrame(columns=['公司名稱', '計畫名稱', '核定日期'])

# for i in range(0,len(df)):
#     if df.iloc[i].values not in DF.values:
#         DF = pd.concat([DF, pd.DataFrame(df.iloc[i]).T], axis = 0)

# DF = DF.sort_values('核定日期', ignore_index=True)
# DF.to_excel(path_update, index = 0)

0
9
16
28
55
65
72
79
88
99
109
115
125
139
145
152
155
160
164
170
172
182
188
198
206
218
228
238
246
254
258
268
278
290
301
311
322
333
345
349
351
359
366
375
386
394
404
419
429
441
451
458
468
474
480
491
498
501
511
523
525
529
531
533
540
550
558
566
578
603
613
623
626
628
632
639
640
650
657
664
667
671
679
690
697
708
716
721
728
734
740
748
756
762
765
767
773
777
782
787
795
803
809
819
826
834
839
848
853
860
865
873
884


In [105]:
# 讀取新的資料
df = pd.read_excel('auto/new_data/4_政府推動計畫名單.xlsx')

# 更新檔案路徑
updatepath = 'auto/data/'
path_update = f'{updatepath}4_政府推動計畫名單.xlsx'

# 嘗試讀取舊資料，若檔案不存在，則建立空的 DataFrame
try:
    DF = pd.read_excel(path_update)
except FileNotFoundError:
    DF = pd.DataFrame(columns=['公司名稱', '計畫名稱', '核定日期'])

# 合併新資料與舊資料，並去除重複資料
DF = pd.concat([DF, df], ignore_index=True).drop_duplicates(subset=['公司名稱', '計畫名稱', '核定日期'])

# 依照 '核定日期' 排序
DF = DF.sort_values(by='核定日期', ignore_index=True)

# 儲存更新後的資料
DF.to_excel(path_update, index=False)


In [106]:
DF

,公司名稱,計畫名稱,核定日期
0,富強鑫機密工業股份有限公司,iSCM-富強鑫AI創鑫應用計畫,2019/01/30
1,芳德鑄鋁股份有限公司,鋁製家具自動化產線與智慧供應鏈建置計畫,2019/01/30
2,聚和國際股份有限公司,精密化學應用生技產業-智造聚和雲供應鏈平台,2019/01/30
3,佳凌科技股份有限公司,精密光學產業高階車載鏡頭智慧供應鏈AI加值應用計畫,2019/01/30
4,路易莎職人咖啡股份有限公司,咖啡職人智慧化供應鏈應用計畫,2019/01/30
...,...,...,...
885,艾飛思科技股份有限公司,Top PCIe V6 64Gbps PAM4測試驗證平台開發計畫,2024/11/01
886,恆昌精密科技股份有限公司,NMP藥液回收蒸餾系統開發計畫,2024/11/01
887,陽程科技股份有限公司,先進半導體封裝狹縫式塗佈製程設備開發與驗證計畫,2024/11/01
888,均華精密工業股份有限公司,面板級封裝固晶製程設備,2024/11/01


In [92]:
df.drop_duplicates()

,公司名稱,計畫名稱,核定日期
0,筑波科技股份有限公司,化合物半導體晶圓品質檢驗技術及機台開發,2024/11/01
1,旺矽科技股份有限公司,矽光子晶圓測試系統開發,2024/11/01
2,天虹科技股份有限公司,矽光子異質封裝之12吋Auto Bonder整機驗證計畫,2024/11/01
3,鴻海精密工業股份有限公司,智載天下：生成式AI賦能智慧城市研訓計畫,2024/11/01
4,艾飛思科技股份有限公司,Top PCIe V6 64Gbps PAM4測試驗證平台開發計畫,2024/11/01
...,...,...,...
885,佳凌科技股份有限公司,精密光學產業高階車載鏡頭智慧供應鏈AI加值應用計畫,2019/01/30
886,聚和國際股份有限公司,精密化學應用生技產業-智造聚和雲供應鏈平台,2019/01/30
887,芳德鑄鋁股份有限公司,鋁製家具自動化產線與智慧供應鏈建置計畫,2019/01/30
888,路易莎職人咖啡股份有限公司,咖啡職人智慧化供應鏈應用計畫,2019/01/30


In [90]:
DF

,公司名稱,計畫名稱,核定日期
0,富強鑫機密工業股份有限公司,iSCM-富強鑫AI創鑫應用計畫,2019/01/30
1,大詠城機械股份有限公司,五軸加工具機底座之治金智慧化生產線系統建置研發計畫,2019/02/19
2,美利達工業股份有限公司,電動輔助全避震登山自行車設計與製程整合之創新技術開發計畫,2019/03/27
3,泓格科技股份有限公司,非侵入式智慧感知系統暨工業網路安全防護技術開發計畫,2019/04/26
4,幸福日子網路股份有限公司,WishDay體驗商孵化募資平台建置計畫,2019/05/13
...,...,...,...
108,群翊工業股份有限公司,智慧化懸吊式自動熱風爐驗證計畫,2024/09/26
109,穩脈科技股份有限公司,MHz/KHz雙輸出之超小型化MEMS振盪器ASIC,2024/10/11
110,永餘智能股份有限公司,智慧充儲系統應用於虛擬電廠平台,2024/10/15
111,友達光電股份有限公司、達擎股份有限公司及達運精密工業股份有限公司,友達智慧亞灣策略暨解決方案研發中心,2024/10/24


In [76]:
pd.DataFrame(df.iloc[i]).T

,公司名稱,計畫名稱,核定日期
1,旺矽科技股份有限公司,矽光子晶圓測試系統開發,2024/11/01


In [61]:
DF

[array([['穩脈科技股份有限公司', 'MHz/KHz雙輸出之超小型化MEMS振盪器ASIC', '2024/10/11'],
        ['矽眾科技股份有限公司', '先進極小尺寸高穩定度車用溫度補償石英振盪器ASIC晶片', '2024/10/11'],
        ['凌陽科技股份有限公司', '多模態大型語言模型加速器晶片滿足生成式AI邊緣應用研發計畫', '2024/10/11'],
        ...,
        ['芯鼎科技股份有限公司', '整合運算擴充晶粒對接技術之車載影像處理駕駛輔助域控制器系統晶片平台研發計畫',
         '2024/10/11'],
        ['耐能智慧股份有限公司', 'Kneron離線生成式AI Edge SoC', '2024/10/11'],
        ['訊灝晶片科技股份有限公司', '後量子加密高效能運算晶片安全計畫', '2024/10/11']], dtype=object),
 array(['筑波科技股份有限公司', '化合物半導體晶圓品質檢驗技術及機台開發', '2024/11/01'], dtype=object)]

In [10]:
DF

,公司名稱,計畫名稱,核定日期,0
0,穩脈科技股份有限公司,MHz/KHz雙輸出之超小型化MEMS振盪器ASIC,2024/10/11,NaN
1,矽眾科技股份有限公司,先進極小尺寸高穩定度車用溫度補償石英振盪器ASIC晶片,2024/10/11,NaN
2,凌陽科技股份有限公司,多模態大型語言模型加速器晶片滿足生成式AI邊緣應用研發計畫,2024/10/11,NaN
3,信驊科技股份有限公司,沉浸式影像AI視覺晶片,2024/10/11,NaN
4,瓦雷科技有限公司,實現大規模AI運算之NVMe-oC記憶體晶片設計,2024/10/11,NaN
...,...,...,...,...
879,耐能智慧股份有限公司,Kneron離線生成式AI Edge SoC,2024/10/11,NaN
880,訊灝晶片科技股份有限公司,後量子加密高效能運算晶片安全計畫,2024/10/11,NaN
公司名稱,NaN,NaN,NaN,筑波科技股份有限公司
計畫名稱,NaN,NaN,NaN,化合物半導體晶圓品質檢驗技術及機台開發
